In [2]:
import scrapy
from scrapy.crawler import CrawlerProcess
from scrapy import Spider
import logging
import pandas as pd
import wget
import json
import os
import time

In [3]:
base_url = 'https://www.wnycstudios.org/shows/radiolab/podcasts/'
pages = range(1,37)

In [4]:
class JsonWriterPipeline(object):

    def open_spider(self, spider):
        self.file = open('quoteresult.jl', 'w')

    def close_spider(self, spider):
        self.file.close()

    def process_item(self, item, spider):
        line = json.dumps(dict(item)) + "\n"
        self.file.write(line)
        return item

In [5]:
class QuotesSpider(scrapy.Spider):
    name = "quotes"
    start_urls = [base_url + str(p) + '/' for p in pages]
    custom_settings = {
        'LOG_LEVEL': logging.WARNING,
        'ITEM_PIPELINES': {'__main__.JsonWriterPipeline': 1}, # Used for pipeline 1
        'FEED_FORMAT':'json',                                 # Used for pipeline 2
        'FEED_URI': 'quoteresult.json'                        # Used for pipeline 2
    }
    
    def parse(self, response):
        yield {
            'links': response.css('a.episode-tease__title-link::attr(href)').extract()
        }

In [6]:
process = CrawlerProcess({
    'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'
})

process.crawl(QuotesSpider)
process.start()

2020-08-16 14:08:14 [scrapy.utils.log] INFO: Scrapy 1.6.0 started (bot: scrapybot)
2020-08-16 14:08:14 [scrapy.utils.log] INFO: Versions: lxml 4.4.1.0, libxml2 2.9.9, cssselect 1.1.0, parsel 1.5.2, w3lib 1.21.0, Twisted 19.10.0, Python 3.7.3 | packaged by conda-forge | (default, Mar 27 2019, 15:43:19) - [Clang 4.0.1 (tags/RELEASE_401/final)], pyOpenSSL 19.0.0 (OpenSSL 1.1.1g  21 Apr 2020), cryptography 2.8, Platform Darwin-19.6.0-x86_64-i386-64bit
2020-08-16 14:08:14 [scrapy.crawler] INFO: Overridden settings: {'FEED_FORMAT': 'json', 'FEED_URI': 'quoteresult.json', 'LOG_LEVEL': 30, 'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'}


In [7]:
ll quoteresult.*

-rw-r--r--@ 1 mac.chelli  staff  16849 Aug 16 14:08 quoteresult.jl
-rw-r--r--  1 mac.chelli  staff  16887 Aug 16 14:08 quoteresult.json


In [8]:
import pandas as pd

In [9]:
results = pd.read_json('quoteresult.jl', lines=True)

In [10]:
def flatten(l):
    flat_list = [item for sublist in l for item in sublist]
    return(flat_list)

In [11]:
podcast_urls = ['https://www.wnycstudios.org' + u + '/' for u in flatten(list(results.links))]

In [12]:
pd.DataFrame(podcast_urls).to_csv('podcast_urls.csv', index = False)

In [13]:
podcast_urls = list(pd.read_csv('podcast_urls.csv')['0'])

In [14]:
podcast_urls

['https://www.wnycstudios.org/podcasts/radiolab/episodes/things/',
 'https://www.wnycstudios.org/podcasts/radiolab/articles/announcement-radiolab-robert-jad/',
 'https://www.wnycstudios.org/podcasts/radiolab/articles/breaking-bongo/',
 'https://www.wnycstudios.org/podcasts/radiolab/articles/breaking-news/',
 'https://www.wnycstudios.org/podcasts/radiolab/articles/dolly-partons-america-neon-moss/',
 'https://www.wnycstudios.org/podcasts/radiolab/articles/songs-cross-borders/',
 'https://www.wnycstudios.org/podcasts/radiolab/articles/birdie-cage/',
 'https://www.wnycstudios.org/podcasts/radiolab/articles/radiolab-presents-dolly-partons-america/',
 'https://www.wnycstudios.org/podcasts/radiolab/articles/silky-love/',
 'https://www.wnycstudios.org/podcasts/radiolab/articles/bliss/',
 'https://www.wnycstudios.org/podcasts/radiolab/articles/asking-another-friend/',
 'https://www.wnycstudios.org/podcasts/radiolab/articles/asking-friend/',
 'https://www.wnycstudios.org/podcasts/radiolab/articl

In [15]:

class DownloadSpider(scrapy.Spider):
    name = "quotes"
    start_urls = podcast_urls
    custom_settings = {
        'LOG_LEVEL': logging.WARNING,
        'ITEM_PIPELINES': {'__main__.JsonWriterPipeline': 1}, # Used for pipeline 1
        'FEED_FORMAT':'json',                                 # Used for pipeline 2
        'FEED_URI': 'quoteresult.json'                        # Used for pipeline 2
    }
    
    def parse(self, response):
        yield {
            'links': response.css('a.download-link::attr(href)').extract_first()
        }

In [16]:
process = CrawlerProcess({
    'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'
})


2020-08-16 14:08:38 [scrapy.utils.log] INFO: Scrapy 1.6.0 started (bot: scrapybot)
2020-08-16 14:08:38 [scrapy.utils.log] INFO: Versions: lxml 4.4.1.0, libxml2 2.9.9, cssselect 1.1.0, parsel 1.5.2, w3lib 1.21.0, Twisted 19.10.0, Python 3.7.3 | packaged by conda-forge | (default, Mar 27 2019, 15:43:19) - [Clang 4.0.1 (tags/RELEASE_401/final)], pyOpenSSL 19.0.0 (OpenSSL 1.1.1g  21 Apr 2020), cryptography 2.8, Platform Darwin-19.6.0-x86_64-i386-64bit


In [17]:
process.crawl(DownloadSpider)
process.start()

2020-08-16 14:08:41 [scrapy.crawler] INFO: Overridden settings: {'FEED_FORMAT': 'json', 'FEED_URI': 'quoteresult.json', 'LOG_LEVEL': 30, 'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'}


ReactorNotRestartable: 

In [18]:
results = pd.read_json('quoteresult.jl', lines = True)

In [21]:
results.to_csv('podcast_download_links.csv', index = False)

In [61]:
for i in results.links:
    if i:
        filename = i.split('/')[-1]
        if os.path.isfile(filename) == False:
            print(filename)
            h = 'http://' + i[i.find('audio.wnyc.org'):]
            time.sleep(5)
            wget.download(h)

radiolab_podcast18unravelingbolero.mp3?aisGetOriginalStream=true
moreperfect052416_episode-zero.mp3?aisGetOriginalStream=true
radiolab_podcast18theprimordialjourney_.mp3?aisGetOriginalStream=true
radiolab_podcast18darksideearth.mp3?aisGetOriginalStream=true
radiolab_podcast18rippinrainbownewer3.mp3?aisGetOriginalStream=true
radiolab_podcast18bonusscavengerhunt.mp3?aisGetOriginalStream=true
radiolab_podcast18tweakthevote.mp3?aisGetOriginalStream=true
radiolab_podcast18badshow.mp3?aisGetOriginalStream=true
radiolab_podcast15nazicamp.mp3?awparams=morgan%3Afalse
radiolab_podcast14theskull.mp3?awparams=postroll%3Afalse
radiolab_podcast13tjdave.mp3
radiolab041811.mp3
radiolab_podcast17ponzi.mp3
radiolab081407pod.mp3
radiolab_podcast11dianerunner.mp3
radiolab_podcast11goalkeeper.mp3
radiolab012511.mp3
radiolab_podcast13quicksand.mp3
radiolab_podcast17crypto.mp3
radiolab_podcast17revisitingfaultline.mp3
radiolab_podcast17gondolier.mp3
radiolab_podcast17theradiolab.mp3
radiolab_podcast17nulland